In [1]:
import numpy as np 
import pandas as pd 
import string
import psycopg2
import config
from sqlalchemy import create_engine
import json



### Extract data from movies_metadata.csv, ratings_small.csv and links.csv into pandas dataframes

In [2]:
# Extract data from movies_metadata.csv
movies_metadata_df = pd.read_csv('Resources/movies_metadata.csv')
movies_metadata_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,10/30/1995,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,12/15/1995,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,12/22/1995,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,12/22/1995,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,2/10/1995,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
# Extract data from ratings_small.csv
ratings_small_df = pd.read_csv('Resources/ratings_small.csv')
#ratings_small_df.head()
ratings_small_df.shape

(100004, 4)

In [4]:
# Extract data from links_small.csv
links_small_df = pd.read_csv('Resources/links_small.csv')
links_small_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


### Clean data and transform it 

##### Cleaning movies_metadata dataframe

In [5]:
# Create a new movies dataframe without the genre column
movies_df = movies_metadata_df[["id", "imdb_id","title","release_date","budget", "revenue" ,"vote_average"]]
movies_df.head()

# Remove the rows with null values
movies_df = movies_df.dropna()
movies_df.shape

# Remove all duplicate rows
movies_df = movies_df.drop_duplicates()
movies_df.shape

# Change the type of id to int
movies_df["id"] =movies_df["id"].astype(int)
movies_df.head()
# Slice first 3 characters of imdb_id to get it in the format of links dataframe
movies_df["imdb_id"] = movies_df["imdb_id"].str[2:]
movies_df.head()

,id,imdb_id,title,release_date,budget,revenue,vote_average
0,862,0114709,Toy Story,10/30/1995,30000000,373554033.0,7.7
1,8844,0113497,Jumanji,12/15/1995,65000000,262797249.0,6.9
2,15602,0113228,Grumpier Old Men,12/22/1995,0,0.0,6.5
3,31357,0114885,Waiting to Exhale,12/22/1995,16000000,81452156.0,6.1
4,11862,0113041,Father of the Bride Part II,2/10/1995,0,76578911.0,5.7


In [6]:
# Create new movies dataframe with genre column
movies_genre_df = movies_metadata_df[["id", "imdb_id","genres","title","release_date","budget", "revenue" ,"vote_average"]]
#movies_df.head()
movies_genre_df.shape

(45463, 8)

In [7]:
# Remove the rows with null values
movies_genre_df = movies_genre_df.dropna()
movies_genre_df.shape


(45362, 8)

In [8]:
# Remove all duplicate rows
movies_genre_df = movies_genre_df.drop_duplicates()
movies_genre_df.shape

(45332, 8)

In [9]:
# Change the type of id to int
movies_genre_df["id"] =movies_genre_df["id"].astype(int)
movies_genre_df.head()

,id,imdb_id,genres,title,release_date,budget,revenue,vote_average
0,862,tt0114709,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,10/30/1995,30000000,373554033.0,7.7
1,8844,tt0113497,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Jumanji,12/15/1995,65000000,262797249.0,6.9
2,15602,tt0113228,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",Grumpier Old Men,12/22/1995,0,0.0,6.5
3,31357,tt0114885,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",Waiting to Exhale,12/22/1995,16000000,81452156.0,6.1
4,11862,tt0113041,"[{'id': 35, 'name': 'Comedy'}]",Father of the Bride Part II,2/10/1995,0,76578911.0,5.7


In [10]:
# Slice first 3 characters of imdb_id to get it in the format of links dataframe
movies_genre_df["imdb_id"] = movies_genre_df["imdb_id"].str[2:]
movies_genre_df.head(10)

,id,imdb_id,genres,title,release_date,budget,revenue,vote_average
0,862,0114709,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,10/30/1995,30000000,373554033.0,7.7
1,8844,0113497,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Jumanji,12/15/1995,65000000,262797249.0,6.9
2,15602,0113228,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",Grumpier Old Men,12/22/1995,0,0.0,6.5
3,31357,0114885,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",Waiting to Exhale,12/22/1995,16000000,81452156.0,6.1
4,11862,0113041,"[{'id': 35, 'name': 'Comedy'}]",Father of the Bride Part II,2/10/1995,0,76578911.0,5.7
5,949,0113277,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",Heat,12/15/1995,60000000,187436818.0,7.7
6,11860,0114319,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",Sabrina,12/15/1995,58000000,0.0,6.2
7,45325,0112302,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Tom and Huck,12/22/1995,0,0.0,5.4
8,9091,0114576,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Sudden Death,12/22/1995,35000000,64350171.0,5.5
9,710,0113189,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",GoldenEye,11/16/1995,58000000,352194034.0,6.6


In [11]:
# The genres column is string, so transform the genres column into json so that we can reference as a list.
# First replace all single quote to double quote to satisfy json format.
movies_genre_df['genres'] = movies_genre_df['genres'].str.replace("'",'"')

# Create an empty list to store json result:
genre_list_name = []
genre_list_temp = []
genre_list_id = []
# Load each row into json format, and store into the list:
for index, row in movies_genre_df.iterrows():
    genre_list_temp = json.loads(row['genres'])
    for item in genre_list_temp:
        genre_list_name.append(item['name'])
        genre_list_id.append(row['id'])
    # create list of dictionaries with the genre name and movie id
genre_list_dict = []
for i in range(len(genre_list_name)):
    genre_list_dict.append({'movie_id': genre_list_id[i],'genre': genre_list_name[i] })
# create a dataframe from the list of dictionaries
genre_list_df = pd.DataFrame(genre_list_dict)
genre_list_df.head()
genre_list_df.reset_index(inplace=True)
genre_list_df = genre_list_df.rename(columns = {'index':'genreId'})
genre_list_df.head()

,genreId,movie_id,genre
0,0,862,Animation
1,1,862,Comedy
2,2,862,Family
3,3,8844,Adventure
4,4,8844,Fantasy


##### Cleaning ratings dataframe

In [12]:
# Create a new dataframe for ratings with only the columns we need
ratings_df = ratings_small_df[["userId", "movieId", "rating"]]
#ratings_df.head()
ratings_df.shape

(100004, 3)

In [13]:
# Remove the rows with null values
ratings_df = ratings_df.dropna()
ratings_df.shape

(100004, 3)

In [14]:
# Remove all duplicate rows
ratings_df = ratings_df.drop_duplicates()
ratings_df.shape

(100004, 3)

In [15]:
# datatype of movieId
ratings_df['movieId'].dtype

dtype('int64')

In [16]:
ratings_df

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0
...,...,...,...
99999,671,6268,2.5
100000,671,6269,4.0
100001,671,6365,4.0
100002,671,6385,2.5


##### Cleaning links dataframe

In [17]:
# Create a new dataframe for links with only the columns we need
links_df = links_small_df[["movieId", "imdbId", "tmdbId"]]
# Drop the rows with null values
links_df.dropna(inplace=True)
# Change the tmdbId to int in the links dataframe
links_df["tmdbId"] = links_df["tmdbId"].apply(np.int64)
links_df

,movieId,imdbId,tmdbId
0,1,114709,862
1,2,113497,8844
2,3,113228,15602
3,4,114885,31357
4,5,113041,11862
...,...,...,...
9120,162672,3859980,402672
9121,163056,4262980,315011
9122,163949,2531318,391698
9123,164977,27660,137608


### Connect to local database

In [21]:
# Connect to your Postgres database
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(config.username, config.password, 'localhost', 5432, 'movies_db'))
con = engine.connect()

In [22]:
# check for tablenames in the database
engine.table_names()

C:\Users\vasan\AppData\Local\Temp/ipykernel_21500/3623636692.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['movies', 'genre', 'ratings', 'links']

In [26]:
# use pandas to load csv converted dataframe into the database
movies_df.to_sql('movies', con, if_exists='replace', index=False)


In [27]:
# confirm that the data was loaded into the database
pd.read_sql_query('select * from movies', con=engine).head()

,id,imdb_id,title,release_date,budget,revenue,vote_average
0,862,0114709,Toy Story,10/30/1995,30000000,373554033.0,7.7
1,8844,0113497,Jumanji,12/15/1995,65000000,262797249.0,6.9
2,15602,0113228,Grumpier Old Men,12/22/1995,0,0.0,6.5
3,31357,0114885,Waiting to Exhale,12/22/1995,16000000,81452156.0,6.1
4,11862,0113041,Father of the Bride Part II,2/10/1995,0,76578911.0,5.7


In [ ]:
# use pandas to load csv converted into dataframe into the database
genre_list_df.to_sql('genre', con, if_exists='replace', index=False)

In [ ]:
# confirm that the data was loaded into the database
pd.read_sql_query('select * from genre', con=engine).head()

In [ ]:
# use pandas to load csv converted into dataframe into the database
ratings_df.to_sql('ratings', con, if_exists='replace', index=False)

In [ ]:
# confirm that the data was loaded into the database
pd.read_sql_query('select * from ratings', con=engine).head()

In [ ]:
# use pandas to load csv converted into dataframe into the database
links_df.to_sql('links', con, if_exists='replace', index=False)

In [ ]:
# confirm that the data was loaded into the database
pd.read_sql_query('select * from links', con=engine).head()